Basic chatbot to act as a clinician to diagnose clinical cases

Load required libraries

see the following URL
https://docs.science.ai.cam.ac.uk/hands-on-llms/2_open_ai/



In [51]:
from openai import OpenAI

In [52]:
import os
import dotenv
from rich import print as rprint

Call chatbot OpenAI API

In [53]:
dotenv.load_dotenv()
# load .env

# get API KEY and store it in variable
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# call OpenAI function
client = OpenAI()


Call API

In [54]:
system_prompt = "You are a clinician and you are diagnosing cases. Given the prompt please give a diagnosis. Please explain all the steps in your reasoning process."
user_query = "A patient has come to you complaining about chest pain. Please give a diagnosis. Please explain all the steps."

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_query}
    ],
    max_tokens=1000 #,
    #temperature=2
)

print(response.choices[0].message.content)


To diagnose a patient presenting with chest pain, I would approach the situation systematically, considering both the potential seriousness of the condition and the detail of the patient's presentation. Here's how I would proceed:

**1. Patient History:**
   - First, I would take a thorough history from the patient, focusing on the character, duration, and severity of the chest pain. Key questions would include:
     - Is the pain sharp, dull, or squeezing?
     - How long does it last? Is it constant or intermittent?
     - What were you doing when the pain started?
     - Does it radiate anywhere (to the arm, jaw, back)?
     - Have you had any previous episodes?
     - Do you have any accompanying symptoms (e.g., sweating, nausea, shortness of breath, dizziness)?

**2. Risk Factors:**
   - Next, I would assess risk factors for cardiac disease, including:
     - Age and sex (e.g., males over 45 and females over 55 are at higher risk).
     - History of hypertension, diabetes, hyperli

Slightly improved version of chatbot with more clinical case information

In [55]:
# read in file with case
f_str = open("master_prompt.txt", "r")
master_prompt = f_str.read()

system_prompt = "You are a clinician and you are diagnosing cases. Given the prompt please give a diagnosis. Please explain all the steps in your reasoning process."
user_query = "A patient has come to you complaining about chest pain. Please give a diagnosis. Please explain all the steps."
user_query = user_query + master_prompt


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_query}
    ],
    max_tokens=1000 #,
    #temperature=2
)

print(response.choices[0].message.content)


Based on the provided clinical case, the diagnosis is **viral pericarditis**, likely due to Coxsackie virus infection. Here is the reasoning process that leads to this diagnosis:

### Step 1: Assess the Symptoms
- The patient is a 34-year-old male presenting with acute chest pain.
- The pain is described as central and persistent for 4 hours, with radiation to both sides of the chest. His previous chest pains lasted only a few seconds and were not alarming in character.
- The current pain is relieved by sitting up and leaning forward, which is characteristic of pericarditis rather than other cardiac causes, such as myocardial infarction (MI) or angina.
- There are no associated symptoms of shortness of breath or palpitations, which also points away from classic cardiac ischemia.

### Step 2: Consider Previous Medical History
- The patient had a recent history of upper respiratory tract infection (URI), suggesting a potential viral etiology.
- The history of occasional stabbing chest pa